<a href="https://colab.research.google.com/github/joaquimor/Poluicao_atmosferica/blob/main/Mensuracao_monoxido_carbono_sentinel5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Your client credentials
client_id = 'sh-e12f32c2-470e-4393-9974-a3a666247241'
client_secret = '0iK6QXHRPrewdgk7iQ6t0Myq94mMEQIb'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

# Parâmetros da requisição
bbox = [-47, -24, -46, -23]
start_date = "2024-10-06"
end_date = "2024-10-07"
collection_id = "sentinel-5p-l2"

# Evalscript para CO (adaptado da documentação)
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["CO", "dataMask"],
    output: {
      bands: 1,
      sampleType: "UINT8"
    }
  };
}

function evaluatePixel(sample) {
  if (sample.dataMask == 0) {
    return [0];
  } else {
    var normalizedCO = (sample.CO - 0.0) / (0.1 - 0.0) * 255;
    return [Math.round(normalizedCO)];
  }
}
"""

# Dados da requisição (adaptado da documentação)
json_request = {
    'input': {
        'bounds': {
            'bbox': bbox,
            'properties': {
                'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'
            }
        },
        'data': [
            {
                'type': collection_id,  # Usando collection_id
                'dataFilter': {
                    'timeRange': {
                        'from': f'{start_date}T00:00:00Z',
                        'to': f'{end_date}T23:59:59Z'
                    },
                    'timeliness': 'NRTI',  # Adicionando timeliness
                },
            }
        ]
    },
    'output': {
        'width': 512,
        'height': 512,
        'responses': [
            {
                'identifier': 'default',
                'format': {
                    'type': 'image/tiff',  # Saída em TIFF
                }
            }
        ]
    },
    'evalscript': evalscript
}

# URL da API do Sentinel Hub
url_request = 'https://sh.dataspace.copernicus.eu/api/v1/process'
headers_request = {
    "Authorization" : "Bearer %s" %token['access_token']
}

# Envie a requisição
response = oauth.request("POST", url_request, headers=headers_request, json=json_request)

# Verifique se a requisição foi bem-sucedida
if response.status_code == 200:
    # Salve os dados de CO em um arquivo GeoTIFF
    with open("co_data.tiff", "wb") as f:
        f.write(response.content)

    print("Dados de CO salvos com sucesso em co_data.tiff.")

    # Visualizar a imagem (opcional)
    image_arr = np.array(Image.open(io.BytesIO(response.content)))
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(image_arr)
    plt.show()

else:
    print(f"Erro ao buscar dados de CO: {response.status_code}")
    print(response.text)